In [6]:
# !matplotlib notebook
import matplotlib.pyplot as plt
from skimage import data
from skimage.exposure import histogram
import tifffile
import os
import sys
from skimage import data, feature, exposure
import numpy as np
from skimage.filters import median, sobel
import napari

path = "E:\\Sudipta\\Arpan"
file_name = "send-1.tif"

In [7]:
def scale_to(x,  dtype):
    r = np.max(x) - np.min(x)
    try:
        t_max = np.iinfo(np.dtype(dtype)).max
    except:
        t_max = np.finfo(np.dtype(dtype)).max
    
    # assert(math.isclose(0,r, abs_tol=np.finfo(float).eps) == False)
    x_s =  ((x - np.min(x)) / r) * t_max
    return x_s.astype(dtype)

def scale_to_float(x,  dtype_in):
#     r = np.max(x) - np.min(x)
    try:
        i_min = np.iinfo(np.dtype(dtype_in)).min
        i_max = np.iinfo(np.dtype(dtype_in)).max
    except:
        i_min = np.finfo(np.dtype(dtype_in)).min
        i_max = np.finfo(np.dtype(dtype_in)).max
    
    r = i_max -i_min
    return (x - i_min)/r
    
    # assert(math.isclose(0,r, abs_tol=np.finfo(float).eps) == False)
#     x_s =  ((x - np.min(x)) / r) * t_max
#     return x_s.astype(dtype)

def subtract(a, b, dtype='uint16'):
    try:
        t_min = np.iinfo(np.dtype(dtype)).min
        t_max = np.iinfo(np.dtype(dtype)).max
    except:
        t_min = np.finfo(np.dtype(dtype)).min
        t_max = np.finfo(np.dtype(dtype)).max        
    return np.clip(a.astype('int32') - b, t_min, t_max).astype(a.dtype)

def top_hat(image, radius=50, light_bg=False):
        from skimage.morphology import white_tophat, black_tophat, disk
        str_el = disk(radius) #you can also use 'ball' here to get a slightly smoother result at the cost of increased computing time
        if light_bg:
            return  scale_to(black_tophat(image, str_el), image.dtype)
        else:
            return  scale_to(white_tophat(image, str_el), image.dtype)

def thresholdV2(image, param=10):
    from skimage.filters import threshold_otsu
    dtype = image.dtype
    thresh = threshold_otsu(image)
    
    return image > thresh

def threshold(image, param=10):
    from skimage.filters import threshold_otsu
    dtype = image.dtype
    thresh = threshold_otsu(image)
    
    return thresh, subtract(image, thresh, image.dtype)

def dspeckle(image, param=10):
    from skimage.filters import median
    dtype = image.dtype
    return median(image)
    

def gaussian_filter(image, param=10):
    from skimage.filters import gaussian
    return scale_to(gaussian(image), image.dtype)


def plot_multi(images):
    fig, ax = plt.subplots(ncols=len(images), figsize=(20, 8))
    for i, image in enumerate(images.items()):
        ax[i].set_title(str(image[0]))
        ax[i].imshow(image[1], clim=[np.min(image[1]), np.max(image[1])], cmap='gray')
    plt.show()

def save_images(images, pre_fix="img"):
    os.makedirs(os.path.join(path,pre_fix), exist_ok=True)
    for key, img in images.items():
        p = os.path.join(path,pre_fix, f"{key}_image.tif")
        print()
        tifffile.imwrite(p, img)

In [8]:
viewer = napari.Viewer()

In [9]:
data = tifffile.imread(os.path.join(path, "send-1.tif"))
image = data[157]
viewer.add_image(image, name="main")

<Image layer 'main' at 0x28f46b94be0>

In [31]:

def get_point2d(image):
    _,threshold_img = threshold(image)
    dspeckle_img = dspeckle(threshold_img)
    gaussian_image = gaussian_filter(dspeckle_img)
    _,gaussian_threshold_img = threshold(gaussian_image)
    top_hat_img = top_hat(gaussian_threshold_img)
    _, top_hat_thres_img = threshold(top_hat_img)
    median_img = scale_to(median(top_hat_thres_img), top_hat_thres_img.dtype)
    return scale_to(sobel(median_img), median_img.dtype)

im = get_point2d(image)
print(im.dtype)
print(np.mean(im))
t, im_th = threshold(im)
print(t)

# viewer.add_image(im, name="result")
# viewer.add_image(im_th, name="result_threshol")
# im_b = im_th > 0
# im_b_t = im_b
# im_b_t = im_b_t.astype('uint8')
# im_b_t[im_b == False] = 0
# im_b_t[im_b == True] = 255
# print(im_b_t[110, 68])
# viewer.add_image(im_b_t, name="binary")
# from scipy import ndimage
# im_b = ndimage.binary_fill_holes(im_b_t).astype('uint8')

# viewer.add_image(im_b, name="mask")

uint16
257.84430298662824
12352
